## Mult-label Image Classification using transformers specific pipeline component

This sample shows how to use `transformers_image_classification_pipeline` component from the `azureml` system registry to fine tune a model for multi-label image classification task using fridgeObjects Dataset. We then deploy the fine tuned model to an online endpoint for real time inference.

### Training data
We will use the [multi-label fridgeObjects](https://cvbp-secondary.z19.web.core.windows.net/datasets/image_classification/multilabelFridgeObjects.zip) dataset.

### Model
We will use the `microsoft-beit-base-patch16-224-pt22k-ft22k` model in this notebook. If you need to fine tune a model that is available on HuggingFace, but not available in `azureml` system registry, you can either register the model and use the registered model or use the `model_name` parameter to instruct the components to pull the model directly from HuggingFace.

### Outline
1. Install dependencies
2. Setup pre-requisites such as compute
3. Pick a model to fine tune
4. Prepare dataset for finetuning the model
5. Submit the fine tuning job using transformers specific image-classification component
6. Review training and evaluation metrics
7. Register the fine tuned model
8. Deploy the fine tuned model for real time inference
9. Test deployed end point
9. Clean up resources

### 1. Install dependencies
Before starting off, if you are running the notebook on Azure Machine Learning Studio or running first time locally, you will need the following packages

In [ ]:
! pip install azure-ai-ml>=1.23.1
! pip install azure-identity==1.13.0

### 2. Setup pre-requisites

#### 2.1 Connect to Azure Machine Learning workspace

Before we dive in the code, you'll need to connect to your workspace. The workspace is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning.

We are using `DefaultAzureCredential` to get access to workspace. `DefaultAzureCredential` should be capable of handling most scenarios. If you want to learn more about other available credentials, go to [set up authentication doc](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk), [azure-identity reference doc](https://learn.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

Replace `AML_WORKSPACE_NAME`, `RESOURCE_GROUP` and `SUBSCRIPTION_ID` with their respective values in the below cell.

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential


experiment_name = (
    "AzureML-Train-Finetune-Vision-MultiLabel-Samples"  # can rename to any valid name
)

credential = DefaultAzureCredential()
workspace_ml_client = None
try:
    workspace_ml_client = MLClient.from_config(credential)
    subscription_id = workspace_ml_client.subscription_id
    resource_group = workspace_ml_client.resource_group_name
    workspace_name = workspace_ml_client.workspace_name
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "SUBSCRIPTION_ID"
    resource_group = "RESOURCE_GROUP"
    workspace_name = "AML_WORKSPACE_NAME"

workspace_ml_client = MLClient(
    credential, subscription_id, resource_group, workspace_name
)

registry_ml_client = MLClient(
    credential,
    subscription_id,
    resource_group,
    registry_name="azureml",
)

#### 2.2 Create compute

In order to finetune a model on Azure Machine Learning studio, you will need to create a compute resource first. **Creating a compute will take 3-4 minutes.** 

For additional references, see [Azure Machine Learning in a Day](https://github.com/Azure/azureml-examples/blob/main/tutorials/azureml-in-a-day/azureml-in-a-day.ipynb). 

##### Create CPU compute for model selection component

In [ ]:
from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError

model_import_cluster_name = "sample-model-import-cluster"
try:
    _ = workspace_ml_client.compute.get(model_import_cluster_name)
    print("Found existing compute target.")
except ResourceNotFoundError:
    print("Creating a new compute target...")
    compute_config = AmlCompute(
        name=model_import_cluster_name,
        type="amlcompute",
        size="Standard_D12_v2",
        idle_time_before_scale_down=120,
        min_instances=0,
        max_instances=4,
    )
    workspace_ml_client.begin_create_or_update(compute_config).result()

##### Create GPU compute for finetune component

The list of GPU machines can be found [here](https://learn.microsoft.com/en-us/azure/virtual-machines/sizes-gpu).

In [ ]:
finetune_cluster_name = "sample-finetune-cluster-gpu"

try:
    _ = workspace_ml_client.compute.get(finetune_cluster_name)
    print("Found existing compute target.")
except ResourceNotFoundError:
    print("Creating a new compute target...")
    compute_config = AmlCompute(
        name=finetune_cluster_name,
        type="amlcompute",
        size="STANDARD_NC6s_v3",
        idle_time_before_scale_down=120,
        min_instances=0,
        max_instances=4,
    )
    workspace_ml_client.begin_create_or_update(compute_config).result()

##### Create GPU compute for model evaluation component

The list of GPU machines can be found [here](https://learn.microsoft.com/en-us/azure/virtual-machines/sizes-gpu).

In [ ]:
# Using the same compute cluster for model evaluation as finetuning. If you want to use a different cluster, specify it below
model_eval_cluster_name = "sample-finetune-cluster-gpu"

try:
    _ = workspace_ml_client.compute.get(model_eval_cluster_name)
    print("Found existing compute target.")
except ResourceNotFoundError:
    print("Creating a new compute target...")
    compute_config = AmlCompute(
        name=model_eval_cluster_name,
        type="amlcompute",
        size="Standard_NC6s_v3",
        idle_time_before_scale_down=120,
        min_instances=0,
        max_instances=4,
    )
    workspace_ml_client.begin_create_or_update(compute_config).result()

### 3. Pick a foundation model to fine tune

We will use the `microsoft-beit-base-patch16-224-pt22k-ft22k` model in this notebook. If you need to fine tune a model that is available on HuggingFace, but not available in `azureml` system registry, you can either register the model and use the registered model or use the `model_name` parameter to instruct the components to pull the model directly from HuggingFace.

Currently following models are supported:

| Model Name | Source |
| ------ | ---------- |
| [microsoft-beit-base-patch16-224-pt22k-ft22k](https://ml.azure.com/registries/azureml/models/microsoft-beit-base-patch16-224-pt22k-ft22k/version/19) | azureml registry |
| [microsoft-swinv2-base-patch4-window12-192-22k](https://ml.azure.com/registries/azureml/models/microsoft-swinv2-base-patch4-window12-192-22k/version/20) | azureml registry |
| [facebook-deit-base-patch16-224](https://ml.azure.com/registries/azureml/models/facebook-deit-base-patch16-224/version/19) | azureml registry |
| [google-vit-base-patch16-224](https://ml.azure.com/registries/azureml/models/google-vit-base-patch16-224/version/17) | azureml registry |
| [Image classification models from Huggingface's Transformer library](https://huggingface.co/models?pipeline_tag=image-classification&library=transformers)| HuggingFace |

In [ ]:
huggingface_model_name = "microsoft/beit-base-patch16-224-pt22k-ft22k"

aml_registry_model_name = "microsoft-beit-base-patch16-224-pt22k-ft22k"
foundation_models = registry_ml_client.models.list(aml_registry_model_name)
foundation_model = max(foundation_models, key=lambda x: int(x.version))
print(
    f"\n\nUsing model name: {foundation_model.name}, version: {foundation_model.version}, id: {foundation_model.id} for fine tuning"
)

### 4. Prepare the dataset for fine-tuning the model

We will use the [multi-label fridgeObjects](https://cvbp-secondary.z19.web.core.windows.net/datasets/image_classification/multilabelFridgeObjects.zip) dataset. The fridge object dataset is annotated in the CSV file, where each image corresponds to a line. It defines a mapping of the filename to the labels. Since this is a multi-label classification problem, each image can be associated to multiple labels.

This is the most common data format for multilabel image classification. Each folder title corresponds to the image label for the images contained inside. 

#### 4.1 Download the Data
We first download and unzip the data locally. By default, the data would be downloaded in `./data` folder in current directory. 
If you prefer to download the data at a different location, update it in `dataset_parent_dir = ...` in the following cell.

In [ ]:
import os
import urllib
from zipfile import ZipFile

# Change to a different location if you prefer
dataset_parent_dir = "./data"

# Create data folder if it doesnt exist.
os.makedirs(dataset_parent_dir, exist_ok=True)

# Download data
download_url = "https://automlsamplenotebookdata-adcuc7f7bqhhh8a4.b02.azurefd.net/image-classification/multilabelFridgeObjects.zip"

# Extract current dataset name from dataset url
dataset_name = os.path.split(download_url)[-1].split(".")[0]
# Get dataset path for later use
dataset_dir = os.path.join(dataset_parent_dir, dataset_name)

# Get the data zip file path
data_file = os.path.join(dataset_parent_dir, f"{dataset_name}.zip")

# Download the dataset
urllib.request.urlretrieve(download_url, filename=data_file)

# Extract files
with ZipFile(data_file, "r") as zip:
    print("extracting files...")
    zip.extractall(path=dataset_parent_dir)
    print("done")
# Delete zip file
os.remove(data_file)

In [ ]:
from IPython.display import Image

sample_image = os.path.join(dataset_dir, "images", "56.jpg")
Image(filename=sample_image)

#### 4.2 Upload the images to Datastore through an AML Data asset (URI Folder)

In order to use the data for training in Azure ML, we upload it to our default Azure Blob Storage of our  Azure ML Workspace.

In [ ]:
# Uploading image files by creating a 'data asset URI FOLDER':

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_data = Data(
    path=dataset_dir,
    type=AssetTypes.URI_FOLDER,
    description="Fridge-items images multilabel",
    name="fridge-items-images-multilabel-classification",
)

uri_folder_data_asset = workspace_ml_client.data.create_or_update(my_data)

print(uri_folder_data_asset)
print("")
print("Path to folder in Blob Storage:")
print(uri_folder_data_asset.path)

#### 4.3 Convert the downloaded data to JSONL

For documentation on preparing the datasets beyond this notebook, please refer to the [documentation on how to prepare datasets](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-prepare-datasets-for-automl-images).

In order to use this data to create an AzureML MLTable, we first need to convert it to the required JSONL format. The following script is creating two `.jsonl` files (one for training and one for validation) in the corresponding MLTable folder. In this example, 20% of the data is kept for validation. For further details on jsonl file used for image classification task in automated ml, please refer to the [data schema documentation for multi-label image classification task](https://learn.microsoft.com/en-us/azure/machine-learning/reference-automl-images-schema#image-classification-multi-label).

In [ ]:
import json
import os

# We will copy each JSONL file within its related MLTable folder
training_mltable_path = os.path.join(dataset_parent_dir, "training-mltable-folder")
validation_mltable_path = os.path.join(dataset_parent_dir, "validation-mltable-folder")

# Create the folders if they don't exist
os.makedirs(training_mltable_path, exist_ok=True)
os.makedirs(validation_mltable_path, exist_ok=True)

train_validation_ratio = 5

# Path to the training and validation files
train_annotations_file = os.path.join(training_mltable_path, "train_annotations.jsonl")
validation_annotations_file = os.path.join(
    validation_mltable_path, "validation_annotations.jsonl"
)

# Baseline of json line dictionary
json_line_sample = {
    "image_url": uri_folder_data_asset.path,
    "label": [],
}

# Path to the labels file
labelFile = os.path.join(dataset_dir, "labels.csv")

# Read each annotation and convert it to jsonl line
with open(train_annotations_file, "w") as train_f:
    with open(validation_annotations_file, "w") as validation_f:
        with open(labelFile, "r") as labels:
            for i, line in enumerate(labels):
                # Skipping the title line and any empty lines.
                if i == 0 or len(line.strip()) == 0:
                    continue
                line_split = line.strip().split(",")
                if len(line_split) != 2:
                    print(f"Skipping the invalid line: {line}")
                    continue
                json_line = dict(json_line_sample)
                json_line["image_url"] += f"images/{line_split[0]}"
                json_line["label"] = line_split[1].strip().split(" ")

                if i % train_validation_ratio == 0:
                    # Validation annotation
                    validation_f.write(json.dumps(json_line) + "\n")
                else:
                    # Train annotation
                    train_f.write(json.dumps(json_line) + "\n")

#### 4.4 Create MLTable data input

Create MLTable data input using the jsonl files created above.

For documentation on creating your own MLTable assets for jobs beyond this notebook, please refer to below resources
- [MLTable YAML Schema](https://learn.microsoft.com/en-us/azure/machine-learning/reference-yaml-mltable) - covers how to write MLTable YAML, which is required for each MLTable asset.
- [Create MLTable data asset](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-data-assets?tabs=Python-SDK#create-a-mltable-data-asset) - covers how to create MLTable data asset. 

In [ ]:
def create_ml_table_file(filename):
    """Create ML Table definition"""

    return (
        "paths:\n"
        "  - file: ./{0}\n"
        "transformations:\n"
        "  - read_json_lines:\n"
        "        encoding: utf8\n"
        "        invalid_lines: error\n"
        "        include_path_column: false\n"
        "  - convert_column_types:\n"
        "      - columns: image_url\n"
        "        column_type: stream_info"
    ).format(filename)


def save_ml_table_file(output_path, mltable_file_contents):
    with open(os.path.join(output_path, "MLTable"), "w") as f:
        f.write(mltable_file_contents)


# Create and save train mltable
train_mltable_file_contents = create_ml_table_file(
    os.path.basename(train_annotations_file)
)
save_ml_table_file(training_mltable_path, train_mltable_file_contents)

# Save train and validation mltable
validation_mltable_file_contents = create_ml_table_file(
    os.path.basename(validation_annotations_file)
)
save_ml_table_file(validation_mltable_path, validation_mltable_file_contents)

### 5. Submit the fine tuning job using `transformers_image_classification_pipeline` component
 
Create the job that uses the `transformers_image_classification_pipeline` component for multi-label image-classification task. Learn more in 5.2 about all the parameters supported for fine tuning.

#### 5.1 Create component

In [ ]:
FINETUNE_PIPELINE_COMPONENT_NAME = "transformers_image_classification_pipeline"
pipeline_component_transformers_func = registry_ml_client.components.get(
    name=FINETUNE_PIPELINE_COMPONENT_NAME, label="latest"
)

#### 5.2 Create arguments to be passed to `transformers_image_classification_pipeline` component

The `transformers_image_classification_pipeline` component consists of model selection and finetuning components. The detailed arguments for each component can be found at following README files:
- [Model Import Component](../../../docs/component_docs/image_finetune/transformers_model_import_component.md)
- [Finetune Component](../../../docs/component_docs/image_finetune/transformers_finetune_component.md)

In [ ]:
deepspeed_config_path = "./deepspeed_configs/zero1.json"
if not os.path.exists(deepspeed_config_path):
    print("DeepSpeed config file not found")
    deepspeed_config_path = None

pipeline_component_args = {
    # # Model import args
    "model_family": "HuggingFaceImage",
    "download_from_source": False,  # True for downloading a model directly from HuggingFace
    "mlflow_model": foundation_model.id,  # foundation_model.id is provided, only foundation_model gives UserErrorException: only path input is supported now but get: ...
    # "model_name": huggingface_model_name, # specify the model_name instead of mlflow_model if you want to use a model from the huggingface hub
    # # Finetune_args
    "image_width": -1,  # Default value is -1 which means it would be overwritten by default image width in Hugging Face feature extractor
    "image_height": -1,  # Default value is -1 which means it would be overwritten by default image height in Hugging Face feature extractor
    "task_name": "image-classification-multilabel",
    "apply_augmentations": True,
    "number_of_workers": 8,
    "deepspeed_config": deepspeed_config_path,
    "auto_find_batch_size": False,
    "precision": "32",
    "random_seed": 42,
    "evaluation_strategy": "epoch",
    "evaluation_steps": 500,
    "logging_strategy": "epoch",
    "logging_steps": 500,
    "save_strategy": "epoch",
    "save_steps": 500,
    "save_total_limit": -1,
    "early_stopping": False,
    "early_stopping_patience": 1,
    "resume_from_checkpoint": False,
    "save_as_mlflow_model": True,
    # # Uncomment one or more lines below to provide specific values, if you wish you override the autoselected default values.
    # "metric_for_best_model": "iou",
    # "number_of_epochs": 15,
    # "max_steps": -1,
    # "training_batch_size": 4,
    # "validation_batch_size": 4,
    # "learning_rate": 5e-5,
    # "learning_rate_scheduler": "warmup_linear",
    # "warmup_steps": 0,
    # "optimizer": "adamw_hf",
    # "weight_decay": 0.0,
    # "gradient_accumulation_step": 1,
    # "label_smoothing_factor": 0.0,
    # "max_grad_norm": 1.0,
    # apply_ort: True,
    # apply_deepspeed: True
    # # Model evaluation args
    # The following parameters map to the dataset fields
    # Uncomment one or more lines below to provide specific values, if you wish you override the autoselected default values.
    # "label_column_name": "label",
    # "input_column_names": "image_url",
}
process_count_per_instance = 1
instance_count = 1

# Ensure that the user provides only one of mlflow_model or model_name
if (
    pipeline_component_args.get("mlflow_model") is None
    and pipeline_component_args.get("model_name") is None
):
    raise ValueError(
        "You must specify either mlflow_model or model_name for the model to finetune"
    )
if (
    pipeline_component_args.get("mlflow_model") is not None
    and pipeline_component_args.get("model_name") is not None
):
    raise ValueError(
        "You must specify ONLY one of mlflow_model and model_name for the model to finetune"
    )
elif (
    pipeline_component_args.get("mlflow_model") is None
    and pipeline_component_args.get("model_name") is not None
):
    use_model_name = huggingface_model_name
elif (
    pipeline_component_args.get("mlflow_model") is not None
    and pipeline_component_args.get("model_name") is None
):
    use_model_name = aml_registry_model_name
print(f"Finetuning model {use_model_name}")

#### 5.3 Utility function to create pipeline using `transformers_image_classification_pipeline` component

In [ ]:
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.entities import PipelineComponent
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes


@pipeline()
def create_pipeline_transformers():
    """Create pipeline."""

    transformers_pipeline_component: PipelineComponent = pipeline_component_transformers_func(
        compute_model_import=model_import_cluster_name,
        compute_finetune=finetune_cluster_name,
        compute_model_evaluation=model_eval_cluster_name,
        training_data=Input(type=AssetTypes.MLTABLE, path=training_mltable_path),
        validation_data=Input(type=AssetTypes.MLTABLE, path=validation_mltable_path),
        # test data
        # Using the same data for validation and test. If you want to use a different dataset for test, specify it below
        test_data=Input(type=AssetTypes.MLTABLE, path=validation_mltable_path),
        instance_count=instance_count,
        process_count_per_instance=process_count_per_instance,
        **pipeline_component_args,
    )
    return {
        # Map the output of the fine tuning job to the output of pipeline job so that we can easily register the fine tuned model. Registering the model is required to deploy the model to an online or batch endpoint.
        "trained_model": transformers_pipeline_component.outputs.mlflow_model_folder,
    }

#### 5.4 Run the fine tuning job using `transformers_image_classification_pipeline` component

In [ ]:
transformers_pipeline_object = create_pipeline_transformers()

# don't use cached results from previous jobs
transformers_pipeline_object.settings.force_rerun = True

# set continue on step failure to False
transformers_pipeline_object.settings.continue_on_step_failure = False

transformers_pipeline_object.display_name = (
    use_model_name + "_transformers_pipeline_component_run_" + "multilabel"
)
# Don't use cached results from previous jobs
transformers_pipeline_object.settings.force_rerun = True

print("Submitting pipeline")

transformers_pipeline_run = workspace_ml_client.jobs.create_or_update(
    transformers_pipeline_object, experiment_name=experiment_name
)

print(f"Pipeline created. URL: {transformers_pipeline_run.studio_url}")

In [ ]:
workspace_ml_client.jobs.stream(transformers_pipeline_run.name)

### 6. Get metrics from finetune component

The model training happens as part of the finetune component. Please follow below steps to extract validation metrics from the run.

##### 6.1 Initialize MLFlow Client

The models and artifacts that are produced by AutoML can be accessed via the MLFlow interface.
Initialize the MLFlow client here, and set the backend as Azure ML, via. the MLFlow Client.

IMPORTANT - You need to have installed the latest MLFlow packages with:

    pip install azureml-mlflow
    pip install mlflow

In [ ]:
import mlflow

# Obtain the tracking URL from MLClient
MLFLOW_TRACKING_URI = workspace_ml_client.workspaces.get(
    name=workspace_ml_client.workspace_name
).mlflow_tracking_uri

print(MLFLOW_TRACKING_URI)

In [ ]:
# Set the MLFLOW TRACKING URI
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
print(f"\nCurrent tracking uri: {mlflow.get_tracking_uri()}")

In [ ]:
from mlflow.tracking.client import MlflowClient

# Initialize MLFlow client
mlflow_client = MlflowClient()

#### 6.2 Get the training and evaluation run

In [ ]:
# Concat 'tags.mlflow.rootRunId=' and pipeline_job.name in single quotes as filter variable
filter = "tags.mlflow.rootRunId='" + transformers_pipeline_run.name + "'"
runs = mlflow.search_runs(
    experiment_names=[experiment_name], filter_string=filter, output_format="list"
)

# Get the training and evaluation runs.
# Using a hacky way till 'Bug 2320997: not able to show eval metrics in FT notebooks - mlflow client now showing display names' is fixed
for run in runs:
    # Check if run.data.metrics.epoch exists
    if "epoch" in run.data.metrics:
        training_run = run
    # Else, check if run.data.metrics.accuracy exists
    elif "iou" in run.data.metrics:
        evaluation_run = run

#### 6.3 Get training metrics

Access the results (such as Models, Artifacts, Metrics) of a previously completed run.

In [ ]:
import pandas as pd

pd.DataFrame(training_run.data.metrics, index=[0]).T

### 7. Register the fine tuned model with the workspace

We will register the model from the output of the fine tuning job. This will track lineage between the fine tuned model and the fine tuning job. The fine tuning job, further, tracks lineage to the foundation model, data and training code.

In [ ]:
import time

# Generating a unique timestamp that can be used for names and versions that need to be unique
timestamp = str(int(time.time()))

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

# Check if the `trained_model` output is available
print(
    f"Pipeline job outputs: {workspace_ml_client.jobs.get(transformers_pipeline_run.name).outputs}"
)

# Fetch the model from pipeline job output - not working, hence fetching from fine tune child job
model_path_from_job = (
    f"azureml://jobs/{transformers_pipeline_run.name}/outputs/trained_model"
)
print(f"Path to register model: {model_path_from_job}")

finetuned_model_name = (
    f"{use_model_name.replace('/', '-')}-fridge-objects-multilabel-classification"
)
finetuned_model_description = f"{use_model_name.replace('/', '-')} fine tuned model for fridge objects multilabel classification"
prepare_to_register_model = Model(
    path=model_path_from_job,
    type=AssetTypes.MLFLOW_MODEL,
    name=finetuned_model_name,
    version=timestamp,  # Use timestamp as version to avoid version conflict
    description=finetuned_model_description,
)
print(f"Prepare to register model: \n{prepare_to_register_model}")

# Register the model from pipeline job output
registered_model = workspace_ml_client.models.create_or_update(
    prepare_to_register_model
)
print(f"Registered model: {registered_model}")

### 8. Deploy the fine tuned model to an online endpoint
Online endpoints give a durable REST API that can be used to integrate with applications that need to use the model.

In [ ]:
import datetime
from azure.ai.ml.entities import ManagedOnlineEndpoint, ManagedOnlineDeployment

# Endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
online_endpoint_name = "hf-ml-fridge-items-" + datetime.datetime.now().strftime(
    "%m%d%H%M"
)
online_endpoint_description = f"Online endpoint for {registered_model.name}, finetuned for fridge objects multilabel classification"
# Create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description=online_endpoint_description,
    auth_mode="key",
    tags={"foo": "bar"},
)
workspace_ml_client.begin_create_or_update(endpoint).result()

In [ ]:
from azure.ai.ml.entities import OnlineRequestSettings, ProbeSettings

deployment_name = "hf-ml-fridge-items-mlflow-deploy"
print(registered_model.id)
print(online_endpoint_name)
print(deployment_name)

# Create a deployment
demo_deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=online_endpoint_name,
    model=registered_model.id,
    instance_type="Standard_DS3_V2",  # Use GPU instance type like STANDARD_NC6s_v3 for faster explanations
    instance_count=1,
    request_settings=OnlineRequestSettings(
        max_concurrent_requests_per_instance=1,
        request_timeout_ms=90000,
        max_queue_wait_ms=500,
    ),
    liveness_probe=ProbeSettings(
        failure_threshold=49,
        success_threshold=1,
        timeout=299,
        period=180,
        initial_delay=180,
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=10,
        success_threshold=1,
        timeout=10,
        period=10,
        initial_delay=2000,
    ),
)

workspace_ml_client.online_deployments.begin_create_or_update(demo_deployment).wait()
endpoint.traffic = {deployment_name: 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

### 9. Test the endpoint with sample data

We will fetch some sample data from the test dataset and submit to online endpoint for inference. We will then display the scored labels alongside the ground truth labels.

In [ ]:
demo_deployment = workspace_ml_client.online_deployments.get(
    name=deployment_name,
    endpoint_name=online_endpoint_name,
)

# Get the details for online endpoint
endpoint = workspace_ml_client.online_endpoints.get(name=online_endpoint_name)

# Existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)
print(demo_deployment)

In [ ]:
# Create request json
import base64
import json

sample_image = os.path.join(dataset_dir, "images", "56.jpg")


def read_image(image_path):
    with open(image_path, "rb") as f:
        return f.read()


request_json = {
    "input_data": [base64.b64encode(read_image(sample_image)).decode("utf-8")],
}

request_file_name = "sample_request_data.json"
with open(request_file_name, "w") as request_file:
    json.dump(request_json, request_file)

In [ ]:
resp = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    deployment_name=demo_deployment.name,
    request_file=request_file_name,
)

In [ ]:
resp

#### Visualize detections
Now that we have scored a test image, we can visualize the prediction for this image.

In [ ]:
! pip install matplotlib
! pip install seaborn

In [ ]:
%matplotlib inline
import json
import numpy as np
import seaborn as sns
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img_np = mpimg.imread(sample_image)
img = Image.fromarray(img_np.astype("uint8"), "RGB")
x, y = img.size
score_threshold = 0.5  # display top objects with confidence score > 0.5

# Set a compact figure size
fig_width = 12
fig_height = 12

# Initialize figure and axes
fig = plt.figure(figsize=(fig_width, fig_height))
gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.2)
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1])
ax1.imshow(img_np)
ax1.axis("off")
prediction_list = json.loads(resp)
prediction = prediction_list[0]

sorted_scores = []
sorted_labels = []
sorted_colors = []

for idx, pred in prediction.items():
    score = pred["score"]
    label = pred["label"]
    if score > score_threshold:
        display_text = f"{label} ({round(score, 3)})"
        print(display_text)

        color = np.random.rand(3)
        sorted_scores.append(score)
        sorted_colors.append(color)
        sorted_labels.append(label)

# Set a stylish color palette
sns.set_palette("pastel")

# Create the bar plot without x-axis and y-axis markings
barplot = sns.barplot(x=sorted_scores, y=sorted_labels, palette=sorted_colors, ax=ax2)
ax2.set_xlabel("")  # Remove x-axis label
ax2.set_ylabel("")  # Remove y-axis label
ax2.set_title(f"Top {len(sorted_scores)} Object Scores", fontsize=12)

# Add scores in front of the bars
for index, value in enumerate(sorted_scores):
    barplot.text(
        value + 0.001, index, f"{value:.2f}", va="center", color="black", fontsize=10
    )

# Remove spines and ticks from the bar plot
barplot.spines["left"].set_visible(False)
barplot.spines["top"].set_visible(False)
barplot.spines["right"].set_visible(False)
barplot.spines["bottom"].set_visible(False)
barplot.tick_params(left=False, top=False, right=False, bottom=False)
barplot.xaxis.set_visible(False)  # Remove x-axis
barplot.yaxis.grid(False)  # Remove y-axis grid

# Customize bar plot lines for a fading effect
for container in barplot.containers:
    for bar in container:
        bar.set_linewidth(0.5)
        bar.set_edgecolor("gray")
        bar.set_alpha(0.7)

# Set plot background color
fig.patch.set_facecolor("#F7F7F7")  # Light gray

plt.tight_layout()
# fig.savefig("plot.png", bbox_inches="tight")
plt.show()

### 10. Clean up resources - delete the online endpoint
Don't forget to delete the online endpoint, else you will leave the billing meter running for the compute used by the endpoint.

In [ ]:
workspace_ml_client.online_endpoints.begin_delete(name=online_endpoint_name).wait()